# 📌 Adım 1 – LibTorch Nedir ve Nasıl Kurulur?

Bu bölümde LibTorch’un ne olduğunu, neden kullanıldığını ve sistemime nasıl kurduğumu adım adım açıklıyorum. Amacım, Python + PyTorch ile eğittiğim modelleri C++ tarafında çalıştırabilecek seviyeye gelmek.

---

## 🔹 LibTorch Nedir?

PyTorch, Python üzerinde yaygın olarak kullanılan bir derin öğrenme kütüphanesidir.  
Bu kütüphanenin C++ tarafındaki resmi karşılığı **LibTorch**’tur.

LibTorch, PyTorch’un çekirdeğini C++ geliştiricilerine açan bir yapıdır.  
Bu sayede:

- Python’da eğittiğim modelleri C++’ta kullanabiliyorum.  
- Gerçek zamanlı sistemlerde (otomotiv, robotik, gömülü cihazlar) düşük gecikme ile inference yapabiliyorum.  
- OpenCV, ROS2, custom C++ kodları ve gerçek zamanlı iş parçacıklarıyla aynı ekosistemde çalışabiliyorum.

Kısacası LibTorch, PyTorch modellerinin üretim ortamında yüksek performansla kullanılmasını sağlıyor.

---

## 🔹 CPU ve GPU (CUDA) Sürümleri Arasındaki Fark

LibTorch indirilirken iki temel paket bulunur:

### **1) CPU Sürümü**
- Sadece işlemci kullanır.  
- Kurulumu kolaydır.  
- CUDA veya özel sürücü gerektirmez.  
- Geliştirme ve test için genellikle yeterlidir.

### **2) CUDA (GPU) Sürümü**
- NVIDIA GPU üzerinden çalışır.  
- Yüksek performanslı inference için uygundur.  
- Doğru CUDA sürümü, doğru NVIDIA driver ve doğru MSVC uyumu gerektirir.  
- Yanlış sürüm kombinasyonları derleme hatalarına yol açabilir.

Bu eğitimde önce **CPU sürümü** ile ilerleyip tüm süreci tamamlıyorum.  
Son aşamada istersem CUDA’ya geçişi de kolayca yapabilirim.

---

## 🔹 LibTorch Kurulum Mantığı (Windows + CMake + MSVC)

Genel kurulum akışı şu şekildedir:

1. PyTorch’un resmi sitesinden LibTorch (C++/Windows) paketini indiriyorum.  
2. Dosyayı örneğin `C:/libs/libtorch` içerisine çıkartıyorum.  
3. CMake tarafında:
   - `find_package(Torch REQUIRED)` ile LibTorch’u projeye ekliyorum.
   - `C:/libs/libtorch/share/cmake/Torch` yolunu CMake’e tanıtıyorum.
   - `target_link_libraries(... ${TORCH_LIBRARIES})` satırıyla derlemeye bağlıyorum.
4. MSVC derleyicisinde `/MD` (Multi-threaded DLL) yapılandırması gerekiyor; LibTorch bu modu kullanıyor.

Bu adımda sadece genel mantığı oturtuyorum.  
CMake tarafındaki dosyaları sonraki adımda adım adım oluşturacağım.

---

## 🔹 İlk Test: LibTorch Doğru Kuruldu Mu?

Kurulumun doğru olup olmadığını kontrol etmek için küçük bir test programı yazıyorum.  
Amaç sadece bir `torch::Tensor` oluşturup ekrana yazdırmak.

```cpp
// Basit LibTorch testi: Tensor oluşturup ekrana yazdırma

#include <iostream>
#include <torch/torch.h>

int main() {
    try {
        torch::Tensor t = torch::tensor({1, 2, 3, 4}, torch::kInt32);
        std::cout << "Tensor: " << t << std::endl;
        std::cout << "Shape: " << t.sizes() << std::endl;
    }
    catch (const c10::Error& e) {
        std::cerr << "LibTorch hatası: " << e.what() << std::endl;
        return -1;
    }
    return 0;
}
```

----

# 📌 Adım 1.1 – LibTorch Kurulumu (Windows)

# 📌 LibTorch Kurulumu (Windows – CPU Sürümü)

Bu bölümde CPU tabanlı LibTorch paketinin Windows ortamına nasıl kurulduğunu adım adım anlatıyorum.  
Amaç, Python + PyTorch ile eğittiğim modelleri C++ tarafında LibTorch kullanarak çalıştırmak (inference) için gerekli kullanım ortamını hazırlamaktır.

---

## 🔹 1) Doğru LibTorch Paketinin Seçilmesi

PyTorch’un resmi sitesine giriyorum:

https://pytorch.org/get-started/locally/

Aşağıdaki seçimleri yapıyorum:

- **Package:** LibTorch  
- **Language:** C++ / Java  
- **Compute Platform:** CPU  

Bu seçim sonucunda indirilen paket genellikle şu isimdedir:
>libtorch-win-shared-with-deps-2.9.1+cpu.zip


Bu paket, GPU/CUDA gerektirmediği için Windows üzerinde en sorunsuz çalışan LibTorch sürümüdür.

---

## 🔹 2) LibTorch Dosyalarının Yerleştirilmesi

İndirdiğim `.zip` dosyasını sistemimde sabit bir konuma çıkarıyorum.

Önerdiğim yol:
>C:/libtorch_cpu

#### Klasör yapısı şu şekilde görünmelidir:
```bash
C:/libtorch_cpu/
bin/
include/
lib/
share/
```
Özellikle şu dosyanın varlığı çok önemlidir:

**C:/libtorch_cpu/share/cmake/Torch/TorchConfig.cmake**

CMake, LibTorch’u bu dosya üzerinden bulmaktadır.

---

## 🔹 3) Ortam Değişkeni (Opsiyonel)

LibTorch’un `bin` klasörünü PATH’e eklemek çalışma zamanındaki DLL hatalarını azaltır ancak zorunlu değildir:

>C:/libtorch_cpu/bin


---

## 🔹 4) Minimal C++ Test Dosyası

Kurulumun doğru yapıldığını kontrol etmek için küçük bir test dosyası hazırlıyorum:

```cpp
#include <iostream>
#include <torch/torch.h>

int main() {
    torch::Tensor t = torch::rand({2, 3});
    std::cout << "Tensor:\n" << t << std::endl;
}
```
**Bu dosya sadece derleyicinin LibTorch’u doğru gördüğünü doğrulamak için kullanılır.**


---

## 🔹 5) CMakeLists.txt Dosyasının Hazırlanması

* CPU LibTorch paketini projeme tanıtmak için minimal CMake yapılandırmasını şu şekilde yazıyorum:
```cpp
cmake_minimum_required(VERSION 3.20 FATAL_ERROR)
project(libtorch_cpu_test)

# LibTorch kurduğum yol
set(CMAKE_PREFIX_PATH "C:/libtorch_cpu")

find_package(Torch REQUIRED)

add_executable(test main.cpp)
target_link_libraries(test "${TORCH_LIBRARIES}")

set_property(TARGET test PROPERTY CXX_STANDARD 17)
```


Buradaki kritik satır:

**set(CMAKE_PREFIX_PATH "C:/libtorch_cpu")**


* Bu sayede CMake, LibTorch’un cmake yapılandırmasına erişebilir.

---
## 🔹 6) CMake ile Derleme

Proje klasörümde sırasıyla:
```bash
mkdir build
cd build
cmake ..
cmake --build .
```


Eğer kurulum doğruysa:

* CMake CPU LibTorch paketini başarıyla bulur

* Proje sorunsuz derlenir

* Çalıştırdığımda rastgele bir tensor ekrana yazdırılır

* Bu aşama sorunsuz tamamlanıyorsa CPU tabanlı LibTorch ortamı doğru şekilde kurulmuş demektir.

----

# 📌 Adım 2 – Simple PyTorch Modeli ve TorchScript’e Export

LibTorch tarafında kullanacağım modelin, önce PyTorch tarafında tanımlanıp TorchScript formatına dönüştürülmesi gerekiyor.  
Bu adımda:

1. Basit bir `SimpleNet` modeli tanımlıyorum  
2. Dummy veri ile forward çalıştırarak modelin çalıştığını test ediyorum  
3. Modeli **TorchScript** formatına çevirip `.pt` dosyası olarak kaydediyorum  

Bu `.pt` dosyası C++ tarafında `torch::jit::load()` ile yüklenecek.


## Model ve Script

```python # SORU 1:

import torch
import torch.nn as nn
import torch.nn.functional as F
import os


class SimpleNet(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=32, num_classes=3):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


model = SimpleNet(input_dim=10, hidden_dim=32, num_classes=3)

dummy_input = torch.randn(1, 10)

with torch.no_grad():
    output = model(dummy_input)

print("Dummy input shape :", dummy_input.shape)
print("Output shape      :", output.shape)
print("Output tensor     :", output)

model.eval()

scripted_model = torch.jit.script(model)
save_path = "model_ts.pt"
scripted_model.save(save_path)
print(f"TorchScript modeli kaydedildi: {os.path.abspath(save_path)}")
```



----
# 📌 Adım 3 – C++ Tarafında TorchScript Modelini Yükleme (model_ts.pt)

Bu adımda Python tarafında oluşturduğum `model_ts.pt` dosyasını C++ + LibTorch kullanarak yüklüyorum ve dummy bir tensor ile `forward` çağrısı yapıyorum.

Amaç:
- LibTorch’un modeli sorunsuz okuyabildiğini
- C++ tarafında `forward` çalıştırabildiğimi
- Çıktı boyutunun Python ile uyumlu olduğunu görmek

Böylece LibTorch + TorchScript hattının temelini doğrulamış oluyorum.

---

## 🔹 1) Proje Yapısı

Basit bir proje yapısı kullanıyorum:

```bash
project_root/
    CMakeLists.txt
    main.cpp
    model_ts.pt
    build/
```

>Not: model_ts.pt dosyasını exe’nin çalışacağı klasörle aynı yere koymak işimi kolaylaştırıyor.
**Aksi durumda torch::jit::load("...") içine tam yolu vermem gerekir.**

## 🔹 2) Model Yükleme ve Dummy Forward – C++ Kodu

Aşağıdaki kodu main.cpp dosyasına yazıyorum.
```cpp
int main() {
    try {
        const std::string model_path = "model_ts.pt";

        torch::jit::script::Module module = torch::jit::load(model_path);

        std::cout << "Model yüklendi: " << model_path << std::endl;

        torch::Device device(torch::kCPU);
        module.to(device);
        module.eval();

        torch::Tensor input = torch::randn({1, 10}, torch::TensorOptions().device(device));

        std::vector<torch::jit::IValue> inputs;
        inputs.push_back(input);
        torch::Tensor output = module.forward(inputs).toTensor();

        std::cout << "Input shape : " << input.sizes() << std::endl;
        std::cout << "Output shape: " << output.sizes() << std::endl;
        std::cout << "Output tensor:\n" << output << std::endl;
    }
    catch (const c10::Error& e) {
        std::cerr << "Model yükleme/çalıştırma hatası: " << e.what() << std::endl;
        return -1;
    }

    return 0;
}
```

# 🔁 PyTorch → TorchScript → C++ (LibTorch) Akışı

Bu projede şu adımları izliyorum:

1. **PyTorch Modeli Yazma**
   - `nn.Module` tabanlı bir `SimpleNet` sınıfı tanımladım.
   - Dummy input ile forward çalıştırarak modelin doğru çalıştığını test ettim.

2. **Modeli TorchScript’e Çevirme**
   - `torch.jit.script(model)` ile modeli TorchScript formatına dönüştürdüm.
   - `scripted_model.save("model_ts.pt")` ile diske kaydettim.
   - Ortaya çıkan `model_ts.pt` dosyası Python’dan bağımsız, C++ tarafından okunabilir hale geldi.

3. **C++ Tarafında Kullanmak İçin Hazırlık**
   - `#include <torch/script.h>` ile TorchScript API’sini kullanacağım C++ dosyasını hazırladım.
   - `torch::jit::load("model_ts.pt")` ile modeli C++ tarafında yükleyecek kodu yazdım.
   - CMake ile LibTorch kütüphanesini projeye bağlayacak yapılandırmayı tasarladım.

LibTorch indirildikten sonra:
- CMake ile projeyi derleyip
- `model_ts.pt` dosyasını gerçekten C++ tarafında yükleyip
- Dummy tensor ile forward çalıştırarak hattın uçtan uca çalıştığını doğrulayacağım.


---